|Name|Date|
|----|----|
|Pushpendra Sharma|2022-04-18|

### Notebook 1 Data_Scrapping

In this notebook, we will scrape tables from pdfs. These pfds were downloaded from TRREB's website.  

#### Table of Contents

1. [Scrapping yearly sales](#yearly)
2. [Scrapping monthly sales](#monthly)


### Why are houses so expensive in Toronto?   

To determine the causes for high prices in Toronto, let's start with economics 101. The price of any good is determined based on the demand and supply of the good in market. To understand the price of housing in market, we can analyze the supply and demand variables behind the house prices.    

The figure below shows different supply and demand variables that can affect the prices in housing market.  
![](data/house_price.png)       

To understand the effect of these variables, data was collected from [Toronto Regional Real Estate Board (TRREB)](https://trreb.ca/index.php/market-news/mls-home-price-index/mls-home-price-index-archive). Data is available in pdf format. This notebook describes the methodology used to scrape data from pdf files. 

In [1]:
import numpy as np
import pandas as pd
#for scrapping tables from pdfs
import tabula

### Scrapping Yearly sales for Toronto   <a name="yearly"></a> 
The yearly sales were extracted from a pdf file that was downloaded form TRREB's website. Following is a snapshot of data present in the pdf file. 
![historic_sales](../references/historic_price.png)

In [3]:
df_list = tabula.read_pdf("../data/raw/historic.pdf", pages='all', multiple_tables=True)

df_list gives a list of data frames from the the table. We can access these dataframes using list indexing. 

In [3]:
#check out the data 
df_list[0].head()

,Year,Sales*,Average Sale Price*,Year.1,Sales*.1,Average Sale Price*.1
0,1976,"19,025","$61,389",1999,"58,957","$228,372"
1,1977,"20,512","$64,559",2000,"58,343","$243,255"
2,1978,"21,184","$67,333",2001,"67,612","$251,508"
3,1979,"23,466","$70,830",2002,"74,759","$275,231"
4,1980,"26,017","$75,694",2003,"78,898","$293,067"


We have to rearrange the data frame to put year, sales and average sales values in corresponding columns.

In [4]:
df_first = df_list[0].iloc[:, :-3]
df_second = df_list[0].iloc[:, -3:]
#change the column names
df_first.columns = ['Year', 'Sale', 'Average_Sale_Price']
df_second.columns = ['Year', 'Sale', 'Average_Sale_Price']

#stack two dataframes
df_yearly_sales = pd.concat([df_first, df_second], ignore_index=True)
df_yearly_sales.head()

,Year,Sale,Average_Sale_Price
0,1976,"19,025","$61,389"
1,1977,"20,512","$64,559"
2,1978,"21,184","$67,333"
3,1979,"23,466","$70,830"
4,1980,"26,017","$75,694"


In [5]:
#save data for later use 
df_yearly_sales.to_csv('../data/processed/yearly_sales.csv')

### Scrapping Monthly house sales by neighborhoods in Toronto <a name="monthly"></a>    
There is one pdf for each month that contains information about monthly sales and MLS home price index. 

The MLS® Home Price Index (HPI) is an index that gives information on a neighborhood's home price levels and trends.

Average or median prices are sensitive to outliers and can vary a lot from one month to another that make it difficult to observe a trend in the housing prices. The MLS® HPI is calculated based on certain attributes that tend to evolve over time.    

Following are two snapshots taken from pdf which show monthly sales and MLS index for different neighborhoods for July 2015.     

Monthly Sales
![monthly_sales](../references/monthly_sales.png)                

Monthly Index 
![monthly_index](../references/monthly_index.png)


To extract data for a specific year, first we will define a function that will return the path of file. This path will be used to import the file and scrape data from it.    

In [8]:
def path_name(folder, year, month):
    """returns a path name of the format data/mw1501.pdf
    Parameters
    ----------
    year: int, year in the filename
    month: int, month in the filename
    folder:string, name of the folder
    Returns
    -------
    complete path for the file
    Example
    -------
    path_name(data, 15, 01) >>> data/mw1501.pdf"""
    
    if month < 10:
        path = folder + '/mw' + str(year) + '0' + str(month) + '.pdf'
    else: 
        path = folder + '/mw' + str(year) + str(month) + '.pdf'
        
    return path

In [9]:
path_name(folder='../data/raw', year=15, month=1)

'../data/raw/mw1501.pdf'

In [7]:
#let's first define a simple function to read the contents of pdf
def read_page_four(folder, start_year, end_year, start_month, end_month,
                   pages, lattice, stream, multiple_tables, pandas_options):
    """to read tables from a pdf file for a given period of time
    Parameters
    ----------
    folder: str, path of the folder where all the files are located
    start_year: int, the starting year for the period
    end_year: int, the ending year for the period
    start_month: int, the starting month for the time period
    end_month: int, the ending month for the time period
    pages: str, iterable, pages  parameter for tabula method
    lattice: bool, parameter for read_pdf method
    stream: bool, parameter for read_pdf method
    multiple_tables: bool, parameter for read_pdf method
    pandas_options: dict, parameter for returned dataframes
    Returns
    -------
    list, list of dataframes"""
    
    df_list = []
    
    #extract data for first year 
    for month in range(start_month, 13):
        path = path_name(folder, start_year, month)
            
        df_first = tabula.read_pdf(path, pages=pages, lattice=lattice, stream=stream,
                                     multiple_tables=multiple_tables, 
                                     pandas_options=pandas_options)[0]
        df_first['year'] = start_year
        df_first['month'] = month
        df_list.append(df_first)
        
    #extract data for all the years except first and last 
    for year in range(start_year+1, end_year):
        for month in range(1, 13):
            #get the path name
            path = path_name(folder, year, month)
            
            df = tabula.read_pdf(path, pages=pages, lattice=lattice, stream=stream,
                                     multiple_tables=multiple_tables, 
                                     pandas_options=pandas_options)[0]
            df['year'] = year
            df['month'] = month
            df_list.append(df)
            
    #extract data for the last year
    for month in range(1, end_month+1):
        path = path_name(folder, end_year, month)
            
        df_last = tabula.read_pdf(path, pages=pages, lattice=lattice, stream=stream,
                                     multiple_tables=multiple_tables, 
                                     pandas_options=pandas_options)[0]
        df_last['year'] = end_year
        df_last['month'] = month
        df_list.append(df_last)
        
    return df_list

Now let's extract monthly sales and housing price between Jan, 2012 to June, 2015 for all the neighborhoods. The monthly sales and price are located on page four in the pdf. 

In [10]:
df_list = read_page_four(folder='../data/raw', start_year=12, end_year=15, start_month=1, end_month=6,
                         pages=4, lattice=False, stream=False, multiple_tables=False,
                         pandas_options={'header': None})

In [11]:
#sanity check - this data should be for 3*12+6=42 months
len(df_list)

42

In [12]:
#check a random dataframe in the list 
df_list[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,NaN,NaN,1,1,1,2,8,3,NaN,4,NaN,5.0,12,1
1,NaN,Number of Sales,Dollar Volume,Average Price,Median Price,New Listings,SNLR (Trend),Active Listings,9Mos. Inv. (Trend),Avg. SP/LP,Avg. DOM,NaN,12,1
2,TREB Total,"4,567","$2,116,961,749","$463,534","$392,000","9,655",60.7%,"12,290",2.2,98%,32,NaN,12,1
3,City of Toronto Total,"1,705","$850,871,140","$499,045","$396,000","4,041",59.6%,"5,134",2.2,99%,31,NaN,12,1
4,Toronto West,433,"$173,892,394","$401,599","$360,000","1,032",58.1%,"1,419",2.6,98%,38,NaN,12,1


We have to remove first two rows and the column (11) that has all the null values. 

In [13]:
#clean out dataframes
for df in df_list:
    df.drop(index=[0,1], inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

In [14]:
#check a random dataframe 
df_list[0].head()

,0,1,2,3,4,5,6,7,8,9,10,year,month
2,TREB Total,"4,567","$2,116,961,749","$463,534","$392,000","9,655",60.7%,"12,290",2.2,98%,32,12,1
3,City of Toronto Total,"1,705","$850,871,140","$499,045","$396,000","4,041",59.6%,"5,134",2.2,99%,31,12,1
4,Toronto West,433,"$173,892,394","$401,599","$360,000","1,032",58.1%,"1,419",2.6,98%,38,12,1
5,Toronto W01,17,"$9,044,400","$532,024","$490,000",71,60.0%,88,2.1,100%,24,12,1
6,Toronto W02,41,"$19,072,811","$465,191","$439,000",67,70.3%,63,1.2,101%,24,12,1


In [15]:
#stack all dataframes
df_price_first = pd.concat(df_list, ignore_index=True)

In [16]:
df_price_first.shape

(1680, 13)

__Note:__ The format of pdf changed over time. The format for the month of July in 2015 was different than previous years. The parameters for read_pdf method are defined based on the format of pdf. Also the output of read_pdf depends on the format of pdf.   

Because of the above reason, I scrapped data in three steps and then joined them.  

In [17]:
df_list_15_19 = read_page_four(folder='../data/raw', start_year=15, end_year=19, start_month=7, end_month=12,
                               pages=4, lattice=False, stream=False, multiple_tables=False,
                               pandas_options={'header': None})

Got stderr: May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:41:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:44 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:44 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:41:45 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:41:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:03 PM org.apache.pdfbox.pdmode

Got stderr: May 04, 2022 5:42:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:16 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:16 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:18 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:42:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:32 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:32 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:34 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:42:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:50 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:50 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:42:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:42:51 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:51 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:42:51 PM org.apache.pdfbox.pdmode

In [18]:
#sanity check - this data should be for 4*12+6=54 months
len(df_list_15_19)

54

In [19]:
df_list_15_19[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,NaN,Number of Sales,Dollar Volume1,Average Price1,Median Price1,New Listings2,SNLR (Trend) 8,Active Listings 3,Mos. Inv. (Trend),9.0,Avg. SP / LP4,Avg. DOM5,15,7
1,TREB Total,"9,880","$6,019,255,672","$609,236","$520,000","14,699",62.7%,"16,673",1.9,NaN,100%,22,15,7
2,City of Toronto Total,"3,480","$2,151,342,405","$618,202","$495,000","5,681",57.9%,"7,335",2.2,NaN,100%,24,15,7
3,Toronto West,894,"$495,489,925","$554,239","$510,000","1,344",59.6%,"1,760",2.2,NaN,100%,25,15,7
4,Toronto W01,77,"$49,859,024","$647,520","$425,000",96,53.3%,130,2.4,NaN,100%,26,15,7


In [20]:
#clean out dataframes
for df in df_list_15_19:
    df.drop(index=[0], inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

In [21]:
df_list_15_19[0].head()

,0,1,2,3,4,5,6,7,8,10,11,year,month
1,TREB Total,"9,880","$6,019,255,672","$609,236","$520,000","14,699",62.7%,"16,673",1.9,100%,22,15,7
2,City of Toronto Total,"3,480","$2,151,342,405","$618,202","$495,000","5,681",57.9%,"7,335",2.2,100%,24,15,7
3,Toronto West,894,"$495,489,925","$554,239","$510,000","1,344",59.6%,"1,760",2.2,100%,25,15,7
4,Toronto W01,77,"$49,859,024","$647,520","$425,000",96,53.3%,130,2.4,100%,26,15,7
5,Toronto W02,83,"$58,385,562","$703,441","$705,000",107,69.7%,100,1.3,102%,18,15,7


In [22]:
df_price_second = pd.concat(df_list_15_19, ignore_index=True)

In [23]:
df_price_second.tail()

,0,1,2,3,4,5,6,7,8,10,11,year,month
2155,Toronto E07,39,"$26,427,759","$677,635","$635,000",22,72.1%,26,1.4,100%,32,19,12
2156,Toronto E08,31,"$22,143,500","$714,306","$713,000",17,59.2%,41,1.9,98%,26,19,12
2157,Toronto E09,52,"$34,040,675","$654,628","$719,500",37,72.1%,25,1.0,104%,14,19,12
2158,Toronto E10,37,"$25,912,620","$700,341","$651,919",22,59.3%,38,2.0,99%,29,19,12
2159,Toronto E11,44,"$23,890,488","$542,966","$597,000",36,68.9%,27,1.3,100%,23,19,12


In [24]:
df_list_20_22 = read_page_four(folder='../data/raw', start_year=20, end_year=22, start_month=1, end_month=3,
                               pages=4, lattice=True, stream=False, multiple_tables=False,
                               pandas_options={'header': None})

In [25]:
#sanity check - this data should be for 2*12+3=27 months
len(df_list_20_22)

27

In [26]:
df_list_20_22[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,Municipality,# of Sales,Dollar Volume1,Average Price1,Median Price1,New Listings2,SNLR (Trend)8,Active Listings3,Mos Inv (Trend)9,Avg. SP/LP4,Avg. LDOM5,NaN,20,1
1,TREB Total,"4,581","$3,845,122,350","$839,363","$735,000","7,836",58.5%,"7,772",2.0,100%,27,37.0,20,1
2,City of Toronto Total,"1,603","$1,417,668,705","$884,385","$725,000","2,633",62.1%,"2,464",1.7,101%,24,33.0,20,1
3,Toronto West,444,"$359,423,612","$809,513","$712,504",648,65.1%,585,1.6,101%,23,32.0,20,1
4,Toronto W01,18,"$17,378,600","$965,478","$715,500",31,74.0%,23,0.9,101%,27,36.0,20,1


In [27]:
#clean out dataframes
for df in df_list_20_22:
    df.drop(index=[0,40], inplace=True)
    df.drop(columns=[10], inplace=True)

In [28]:
df_list_20_22[0].head()

,0,1,2,3,4,5,6,7,8,9,11,year,month
1,TREB Total,"4,581","$3,845,122,350","$839,363","$735,000","7,836",58.5%,"7,772",2.0,100%,37.0,20,1
2,City of Toronto Total,"1,603","$1,417,668,705","$884,385","$725,000","2,633",62.1%,"2,464",1.7,101%,33.0,20,1
3,Toronto West,444,"$359,423,612","$809,513","$712,504",648,65.1%,585,1.6,101%,32.0,20,1
4,Toronto W01,18,"$17,378,600","$965,478","$715,500",31,74.0%,23,0.9,101%,36.0,20,1
5,Toronto W02,29,"$31,431,000","$1,083,828","$1,007,000",36,69.2%,24,1.0,101%,32.0,20,1


In [29]:
df_price_third = pd.concat(df_list_20_22, ignore_index=True)

In [30]:
df_price_third.tail()

,0,1,2,3,4,5,6,7,8,9,11,year,month
1048,Toronto E06,50,"$70,984,198","$1,419,684","$1,251,550",65,64.5%,33,1.0,110%,NaN,22,3
1049,Toronto E07,89,"$93,585,728","$1,051,525","$958,888",171,74.2%,89,0.7,122%,NaN,22,3
1050,Toronto E08,92,"$102,308,330","$1,112,047","$1,136,000",122,72.9%,59,0.8,122%,NaN,22,3
1051,Toronto E09,93,"$95,114,005","$1,022,731","$1,030,000",169,77.2%,82,0.6,122%,NaN,22,3
1052,Toronto E10,90,"$113,712,137","$1,263,468","$1,290,000",124,75.1%,36,0.7,122%,NaN,22,3


In [31]:
columns = ['Name', 'Sales', 'Dollar Volume', 'Average Price', 'Median Price', 'New listings', 'SNLR',
          'Active listings', 'MI', 'Avg SP/LP', 'Avg DOM', 'year', 'month']

In [32]:
#change column names for all the dataframes 
df_price_first.columns = columns
df_price_second.columns = columns
df_price_third.columns = columns

In [33]:
df_price_third.head()

,Name,Sales,Dollar Volume,Average Price,Median Price,New listings,SNLR,Active listings,MI,Avg SP/LP,Avg DOM,year,month
0,TREB Total,"4,581","$3,845,122,350","$839,363","$735,000","7,836",58.5%,"7,772",2.0,100%,37.0,20,1
1,City of Toronto Total,"1,603","$1,417,668,705","$884,385","$725,000","2,633",62.1%,"2,464",1.7,101%,33.0,20,1
2,Toronto West,444,"$359,423,612","$809,513","$712,504",648,65.1%,585,1.6,101%,32.0,20,1
3,Toronto W01,18,"$17,378,600","$965,478","$715,500",31,74.0%,23,0.9,101%,36.0,20,1
4,Toronto W02,29,"$31,431,000","$1,083,828","$1,007,000",36,69.2%,24,1.0,101%,32.0,20,1


In [34]:
#combine three dataframes into one dataframe
df_price = pd.concat([df_price_first, df_price_second, df_price_third], ignore_index=True)
df_price.head()

,Name,Sales,Dollar Volume,Average Price,Median Price,New listings,SNLR,Active listings,MI,Avg SP/LP,Avg DOM,year,month
0,TREB Total,"4,567","$2,116,961,749","$463,534","$392,000","9,655",60.7%,"12,290",2.2,98%,32,12,1
1,City of Toronto Total,"1,705","$850,871,140","$499,045","$396,000","4,041",59.6%,"5,134",2.2,99%,31,12,1
2,Toronto West,433,"$173,892,394","$401,599","$360,000","1,032",58.1%,"1,419",2.6,98%,38,12,1
3,Toronto W01,17,"$9,044,400","$532,024","$490,000",71,60.0%,88,2.1,100%,24,12,1
4,Toronto W02,41,"$19,072,811","$465,191","$439,000",67,70.3%,63,1.2,101%,24,12,1


In [35]:
df_price.shape

(4893, 13)

In [36]:
#export the price data for later use
df_price.to_csv('../data/processed/housing_price.csv')

In [37]:
df_price.tail()

,Name,Sales,Dollar Volume,Average Price,Median Price,New listings,SNLR,Active listings,MI,Avg SP/LP,Avg DOM,year,month
4888,Toronto E06,50,"$70,984,198","$1,419,684","$1,251,550",65,64.5%,33,1.0,110%,NaN,22,3
4889,Toronto E07,89,"$93,585,728","$1,051,525","$958,888",171,74.2%,89,0.7,122%,NaN,22,3
4890,Toronto E08,92,"$102,308,330","$1,112,047","$1,136,000",122,72.9%,59,0.8,122%,NaN,22,3
4891,Toronto E09,93,"$95,114,005","$1,022,731","$1,030,000",169,77.2%,82,0.6,122%,NaN,22,3
4892,Toronto E10,90,"$113,712,137","$1,263,468","$1,290,000",124,75.1%,36,0.7,122%,NaN,22,3


So far we have extracted monthly sales and house prices between the year 2012 and 2022. Now let's extract MLS index for each of the neighborhoods over the same period of time.    

This data is located on the page 26 of each of the pdf. It was noted that Jan 2012 did not have MLS index.       

The format of tables change over time therefore data was extracted over four time periods and then combined. 

In [38]:
df_MLS_12_15 = read_page_four(folder='../data/raw', start_year=12, end_year=15, start_month=2, end_month=6,
                               pages=26, lattice=False, stream=True, multiple_tables=False,
                               pandas_options={'header': None})

In [39]:
# sanity check - this data should be for 3*12 + 5 = 41 months
len(df_MLS_12_15)

41

In [40]:
#check out data 
df_MLS_12_15[0].head()

,0,1,2,3,4,5,6,7,8,9,10,year,month
0,NaN,NaN,Composite,NaN,Single-Family Detached,NaN,Single-Family Attached,NaN,Townhouse,NaN,Apartment,12,2
1,NaN,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,12,2
2,TREB Total,145.6,"$447,400 7.30%",144.7,"$544,800 8.39%",149.2,"$426,900 8.51%",142.0,"$323,500 6.13%",143.7,"$303,300 4.06%",12,2
3,City of Toronto,150.5,"$482,600 6.29%",151.6,"$660,000 8.36%",158.5,"$529,300 9.31%",146.5,"$370,200 5.02%",146.7,"$320,100 3.31%",12,2
4,Toronto W01,159.2,"$567,800 7.42%",160.1,"$840,100 12.43%",165.2,"$632,500 9.84%",194.8,"$351,500 17.92%",140.6,"$322,800 -3.63%",12,2


In [41]:
#remove first two rows for each dataframes 
for df in df_MLS_12_15:
    df.drop(index=[0,1], inplace=True)

In [42]:
#check a random dataframe for snaity check 
df_MLS_12_15[20].head()

,0,1,2,3,4,5,6,7,8,9,10,year,month
2,TREB Total,156.6,"$477,700 4.54%",156.8,"$583,100 5.02%",161.6,"$456,500 4.53%",153.2,"$335,500 4.57%",149.9,"$301,100 2.74%",13,10
3,City of Toronto,159.6,"$523,100 3.84%",162.8,"$705,800 4.90%",169.7,"$562,800 3.98%",156.3,"$378,000 3.99%",152.4,"$314,700 2.42%",13,10
4,Toronto W01,155.7,"$635,400 -4.18%",153.8,"$792,700 -5.06%",161.1,"$621,600 -5.95%",210.7,"$433,900 1.40%",138.9,"$324,100 -1.91%",13,10
5,Toronto W02,182.7,"$659,000 6.59%",180.1,"$739,200 6.13%",206.4,"$632,300 7.22%",140.0,"$386,000 4.32%",124.8,"$518,700 10.05%",13,10
6,Toronto W03,164.3,"$424,400 5.52%",167.4,"$455,300 7.10%",169.5,"$434,300 4.57%",-,- -,126.8,"$235,200 -0.08%",13,10


In [43]:
df_MLS_first = pd.concat(df_MLS_12_15, ignore_index=True)

In [44]:
df_MLS_first.tail()

,0,1,2,3,4,5,6,7,8,9,10,year,month
1512,Toronto E07,198.1,"$488,800 10.36%",225.5,"$738,000 13.49%",217.5,"$568,500 13.34%",200.1,"$433,500 12.86%",165.1,"$282,700 5.83%",15,6
1513,Toronto E08,183.3,"$450,200 10.82%",195.4,"$610,000 9.04%",162.4,"$417,100 0.87%",177.8,"$363,000 7.95%",152.3,"$243,100 10.76%",15,6
1514,Toronto E09,176.5,"$426,200 10.38%",195.0,"$555,000 12.13%",179.1,"$439,500 9.94%",178.3,"$328,600 14.29%",154.5,"$289,800 7.22%",15,6
1515,Toronto E10,191.1,"$541,000 12.68%",191.8,"$614,000 12.03%",184.4,"$479,600 9.70%",199.3,"$349,100 15.00%",145.7,"$234,800 10.80%",15,6
1516,Toronto E11,182.2,"$401,700 14.16%",203.4,"$564,900 13.76%",198.6,"$450,200 15.67%",151.4,"$297,500 18.19%",144.5,"$215,700 9.72%",15,6


In [45]:
df_MLS_15_19 = read_page_four(folder='../data/raw', start_year=15, end_year=19, start_month=7, end_month=12,
                               pages=26, lattice=False, stream=True, multiple_tables=False,
                               pandas_options={'header': None})

Got stderr: May 04, 2022 5:48:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:48:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:48:59 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:48:59 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:01 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:01 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:02 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:49:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:18 PM org.apache.pdfbox.pdmode

Got stderr: May 04, 2022 5:49:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:30 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:30 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:30 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:30 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:31 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:49:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:42 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:42 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:44 PM org.apache.pdfbox.pdmodel.font.PDT

Got stderr: May 04, 2022 5:49:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:57 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:57 PM org.apache.pdfbox.pdmodel.PDDocument importPage
May 04, 2022 5:49:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 04, 2022 5:49:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 04, 2022 5:49:58 PM org.apache.pdfbox.pdmode

In [46]:
# sanity check - this data should be 3*12 + 6 + 12 = 54 months
len(df_MLS_15_19)

54

In [47]:
df_MLS_15_19[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,NaN,NaN,Composite,NaN,Single-Family Detached,NaN,Single-Family Attached,NaN,Townhouse,NaN,Apartment,NaN,15,7
1,NaN,Index,Benchmark,Yr./Yr. % Chg.,Index Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,15,7
2,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,"$326,600",3.90%,15,7
3,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,"$341,800",3.83%,15,7
4,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,"$332,100",0.99%,15,7


In [48]:
# remove first two rows for each dataframe 
for df in df_MLS_15_19:
    df.drop(index=[0,1], inplace=True)

In [49]:
#check a random dataframe for snaity check 
df_MLS_15_19[22].tail()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
34,Toronto E07,268,"$656,000",22.58%,"305 $999,700 19.80%",296,"$774,500 23.73%",275,"$595,800 20.09%",230,"$394,200",26.34%,17,5
35,Toronto E08,263,"$634,200",32.59%,"276 $858,700 25.91%",243,"$629,600 25.72%",246,"$502,500 33.73%",245,"$390,700",48.69%,17,5
36,Toronto E09,251,"$598,900",27.81%,"275 $780,300 21.59%",260,"$635,900 22.18%",278,"$517,200 37.36%",224,"$419,800",36.21%,17,5
37,Toronto E10,285,"$799,700",33.63%,"282 $902,000 31.05%",271,"$702,200 32.42%",322,"$562,300 44.12%",212,"$341,700",38.47%,17,5
38,Toronto E11,262,"$572,400",30.59%,"288 $801,600 27.69%",280,"$633,700 28.47%",198,"$389,400 15.21%",255,"$381,100",48.66%,17,5


In [50]:
df_MLS_second = pd.concat(df_MLS_15_19, ignore_index=True)

In [51]:
print(df_MLS_second.shape)
df_MLS_second.head()

(1998, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,"$326,600",3.90%,15,7
1,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,"$341,800",3.83%,15,7
2,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,"$332,100",0.99%,15,7
3,Toronto W02,195,"$700,900",2.86%,"201 $824,200 3.67%",226,"$691,400 3.34%",156,"$431,200 0.71%",141,"$571,500",4.53%,15,7
4,Toronto W03,202,"$523,600",10.75%,"204 $554,800 9.38%",213,"$544,400 13.21%",-,- -,143,"$272,500",6.48%,15,7


In [52]:
df_MLS_20_21 = read_page_four(folder='../data/raw', start_year=20, end_year=21, start_month=1, end_month=2,
                               pages=26, lattice=True, stream=True, multiple_tables=False,
                               pandas_options={'header': None})

In [53]:
# sanity check - this data should be 1*12 + 2 = 14 months
len(df_MLS_20_21)

14

In [54]:
#check out a random dataframe
df_MLS_20_21[9].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,year,month
0,NaN,Composite,Single Family Detached,Single Family Attached,Townhouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,10
1,NaN,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,NaN,20,10
2,TREB Total,295.8,"$897,700",10.79%,290.6,"$1,073,200",12.68%,301.2,"$846,400",12.64%,308.9,"$678,600",12.78%,291.9,"$585,100",6.15%,20,10
3,City of Toronto,296.9,"$962,700",7.18%,288.8,"$1,267,200",10.69%,305.1,"$1,016,900",9.39%,305.8,"$728,700",10.96%,296.7,"$611,100",4.55%,20,10
4,Toronto W01,276.0,"$1,112,300",3.49%,263.5,"$1,375,000",4.77%,281.2,"$1,084,300",4.42%,286.8,"$607,000",9.55%,278.8,"$630,300",2.16%,20,10


In [55]:
# remove first two rows and last row for each dataframe 
for df in df_MLS_20_21:
    df.drop(index=[0, 1, 38], inplace=True)

In [56]:
#check a random dataframe for snaity check 
df_MLS_20_21[5].tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,year,month
33,Toronto E06,284.2,"$792,500",3.68%,289.8,"$827,800",6.08%,291.9,"$694,200",5.72%,-,-,-,254.1,"$537,400",-6.85%,20,6
34,Toronto E07,300.8,"$737,300",12.62%,292.0,"$957,100",8.23%,285.9,"$747,500",10.17%,293.1,"$635,000",10.44%,309.7,"$530,400",15.78%,20,6
35,Toronto E08,300.3,"$724,700",8.06%,275.3,"$858,100",5.72%,249.6,"$647,100",8.38%,305.4,"$623,300",7.50%,344.9,"$550,200",11.76%,20,6
36,Toronto E09,291.3,"$696,400",10.89%,270.8,"$768,900",6.07%,264.4,"$646,600",7.70%,340.6,"$633,000",15.61%,305.0,"$572,100",14.58%,20,6
37,Toronto E10,299.7,"$839,900",12.71%,274.4,"$879,200",8.42%,280.8,"$728,100",13.00%,385.6,"$673,200",21.56%,303.9,"$490,300",14.94%,20,6


In [57]:
df_MLS_third = pd.concat(df_MLS_20_21, ignore_index=True)

In [58]:
df_MLS_third.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,year,month
499,Toronto E06,327.6,"$913,500",17.38%,342.0,"$976,900",21.58%,341.5,"$812,200",21.44%,-,-,-,261.2,"$552,400",-1.10%,21,2
500,Toronto E07,313.3,"$768,000",7.22%,315.1,"$1,032,800",9.49%,303.7,"$794,100",10.96%,312.1,"$676,100",12.47%,312.7,"$535,500",3.92%,21,2
501,Toronto E08,331.8,"$800,700",15.73%,321.4,"$1,001,800",20.19%,292.9,"$759,300",22.81%,327.0,"$667,400",11.72%,353.6,"$564,100",10.12%,21,2
502,Toronto E09,316.2,"$756,000",10.60%,319.2,"$906,300",16.07%,312.8,"$765,000",16.76%,340.7,"$633,100",9.20%,310.1,"$581,700",5.55%,21,2
503,Toronto E10,328.3,"$920,100",17.38%,316.0,"$1,012,500",19.29%,317.1,"$822,300",20.89%,378.6,"$661,000",14.59%,302.8,"$488,500",9.55%,21,2


In [59]:
df_MLS_21_22 = read_page_four(folder='../data/raw', start_year=21, end_year=22, start_month=3, end_month=3,
                               pages=26, lattice=True, stream=True, multiple_tables=False,
                               pandas_options={'header': None})

In [60]:
# sanity check - this data should be 10 + 3 = 14 months
len(df_MLS_21_22)

13

In [61]:
#check out a random dataframe
df_MLS_21_22[2].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,year,month
0,NaN,Composite,Single Family Detached,Single Family Attached,Townhouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,5
1,NaN,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,Yr./Yr. % Chg.,Index,Benchmark,21,5
2,TRREB Total,342.3,"$1,045,800",18.81%,344.7,"$1,289,600",25.07%,356.3,"$1,003,700",23.37%,342.9,"$738,100",19.60%,322.6,"$638,900",21,5
3,City of Toronto,335.4,"$1,105,900",10.66%,342.7,"$1,548,400",18.91%,362.3,"$1,224,000",16.87%,330.5,"$783,900",12.45%,323.7,"$660,400",21,5
4,Toronto W01,323.5,"$1,380,900",11.13%,344.0,"$1,916,100",12.42%,368.1,"$1,463,100",13.96%,276.3,"$831,300",10.26%,309.9,"$700,700",21,5


In [62]:
# remove first two rows for each dataframe 
for df in df_MLS_21_22:
    df.drop(index=[0, 1], inplace=True)

In [63]:
#check a random dataframe for snaity check 
df_MLS_21_22[5].tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,year,month
33,Toronto E06,343.0,"$953,900",10.86%,350.7,"$1,001,800",11.47%,347.4,"$826,200",10.29%,325.9,"$720,200",11.65%,308.6,"$635,300",21,8
34,Toronto E07,347.9,"$852,800",14.07%,351.6,"$1,152,400",16.15%,345.1,"$902,300",16.63%,353.5,"$765,800",16.86%,341.8,"$585,300",21,8
35,Toronto E08,347.5,"$801,100",13.56%,331.9,"$1,034,500",13.08%,297.7,"$771,800",11.83%,320.6,"$587,800",16.29%,382.1,"$521,300",21,8
36,Toronto E09,340.2,"$812,300",13.70%,337.1,"$957,100",16.93%,324.2,"$792,900",14.32%,364.4,"$656,000",17.70%,341.1,"$639,800",21,8
37,Toronto E10,357.3,"$1,002,000",20.02%,343.7,"$1,101,200",22.31%,350.1,"$907,800",24.02%,402.9,"$657,200",17.84%,336.5,"$542,900",21,8


In [64]:
df_MLS_fourth = pd.concat(df_MLS_21_22, ignore_index=True)

In [65]:
df_MLS_fourth.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,year,month
0,TRREB Total,331.9,"$1,007,600",16.54%,337.5,"$1,246,900",23.58%,348.3,"$978,500",21.91%,338.1,"$742,800",16.51%,303.1,"$607,500",21,3
1,City of Toronto,317.4,"$1,029,200",7.45%,325.0,"$1,426,100",17.84%,339.8,"$1,132,600",15.74%,317.4,"$756,300",9.71%,306.0,"$630,300",21,3
2,Toronto W01,299.6,"$1,207,400",10.55%,307.2,"$1,603,000",18.75%,326.0,"$1,257,100",18.33%,290.2,"$614,200",5.57%,291.7,"$659,500",21,3
3,Toronto W02,357.2,"$1,269,700",13.50%,347.4,"$1,469,500",18.73%,387.3,"$1,187,200",18.77%,374.9,"$723,200",4.55%,334.4,"$717,800",21,3
4,Toronto W03,370.9,"$956,000",18.80%,382.4,"$1,039,600",20.82%,373.0,"$955,400",21.34%,266.0,"$644,800",5.06%,328.9,"$562,000",21,3


In [66]:
# check all the dataframes to concat
display(df_MLS_first.head())
display(df_MLS_second.head())
display(df_MLS_third.head())
display(df_MLS_fourth.head())

,0,1,2,3,4,5,6,7,8,9,10,year,month
0,TREB Total,145.6,"$447,400 7.30%",144.7,"$544,800 8.39%",149.2,"$426,900 8.51%",142.0,"$323,500 6.13%",143.7,"$303,300 4.06%",12,2
1,City of Toronto,150.5,"$482,600 6.29%",151.6,"$660,000 8.36%",158.5,"$529,300 9.31%",146.5,"$370,200 5.02%",146.7,"$320,100 3.31%",12,2
2,Toronto W01,159.2,"$567,800 7.42%",160.1,"$840,100 12.43%",165.2,"$632,500 9.84%",194.8,"$351,500 17.92%",140.6,"$322,800 -3.63%",12,2
3,Toronto W02,163.0,"$572,400 13.67%",161.4,"$664,800 11.54%",182.5,"$557,600 17.14%",128.1,"$366,800 4.83%",110.3,"$326,600 -1.52%",12,2
4,Toronto W03,146.9,"$375,700 6.06%",147.8,"$403,900 7.26%",150.0,"$386,100 5.71%",-,- -,131.9,"$216,300 -7.83%",12,2


,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,"$326,600",3.90%,15,7
1,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,"$341,800",3.83%,15,7
2,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,"$332,100",0.99%,15,7
3,Toronto W02,195,"$700,900",2.86%,"201 $824,200 3.67%",226,"$691,400 3.34%",156,"$431,200 0.71%",141,"$571,500",4.53%,15,7
4,Toronto W03,202,"$523,600",10.75%,"204 $554,800 9.38%",213,"$544,400 13.21%",-,- -,143,"$272,500",6.48%,15,7


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,year,month
0,TREB Total,273.0,"$828,200",8.72%,262.9,"$970,700",7.39%,273.1,"$767,800",7.73%,280.4,"$619,000",8.81%,282.6,"$564,000",11.26%,20,1
1,City of Toronto,281.8,"$913,700",8.76%,265.1,"$1,163,300",6.81%,281.6,"$938,700",6.99%,278.0,"$661,800",4.71%,289.7,"$595,000",10.61%,20,1
2,Toronto W01,264.1,"$1,064,400",6.53%,249.6,"$1,302,400",5.09%,267.8,"$1,032,600",5.60%,271.9,"$575,500",7.64%,268.0,"$605,900",7.37%,20,1
3,Toronto W02,296.0,"$1,053,400",5.60%,283.4,"$1,198,800",5.20%,314.1,"$962,700",1.19%,276.7,"$652,000",2.67%,306.5,"$657,900",12.81%,20,1
4,Toronto W03,302.4,"$778,300",6.89%,304.5,"$827,800",6.92%,297.2,"$761,200",6.64%,267.5,"$545,700",1.10%,303.3,"$518,200",10.94%,20,1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,year,month
0,TRREB Total,331.9,"$1,007,600",16.54%,337.5,"$1,246,900",23.58%,348.3,"$978,500",21.91%,338.1,"$742,800",16.51%,303.1,"$607,500",21,3
1,City of Toronto,317.4,"$1,029,200",7.45%,325.0,"$1,426,100",17.84%,339.8,"$1,132,600",15.74%,317.4,"$756,300",9.71%,306.0,"$630,300",21,3
2,Toronto W01,299.6,"$1,207,400",10.55%,307.2,"$1,603,000",18.75%,326.0,"$1,257,100",18.33%,290.2,"$614,200",5.57%,291.7,"$659,500",21,3
3,Toronto W02,357.2,"$1,269,700",13.50%,347.4,"$1,469,500",18.73%,387.3,"$1,187,200",18.77%,374.9,"$723,200",4.55%,334.4,"$717,800",21,3
4,Toronto W03,370.9,"$956,000",18.80%,382.4,"$1,039,600",20.82%,373.0,"$955,400",21.34%,266.0,"$644,800",5.06%,328.9,"$562,000",21,3


There should be 17 columns in each of the dataframe. The first dataframe contains only 12 columns. This is because some of the columns in this dataframe contain information about two columns. For example, columns `2`, `4`, `6`, `8`, and `10` contain information for two columns. We have to extract this information into two columns.               

The second data frame contains 13 columns because of the same reason as of first data frame.   

The third data frame contain 17 columns.              

The fourth dataframe is missing the last columns. This happened due to `tabula` library's inability to extract all the columns from pdf. The 17 column names for each of the dataframe are given below in the cell.   

In [67]:
columns_MLS = ['Name', 'index_Cp', 'benchmark_Cp', 'yrtoyr_chg_Cp',
              'index_SFD', 'benchmark_SFD', 'yrtoyr_chg_SFD',
              'index_SFA', 'benchmark_SFA', 'yrtoyr_chg_SFA',
              'index_Th', 'benchmark_Th', 'yrtoyr_chg_Th',
              'index_Ap', 'benchmark_Ap', 'yrtoyr_chg_Ap', 'year', 'month']

Let's clean the first dataframe to get 17 columns.   

In [68]:
df_MLS_first[['benchmark_Cp', 'yrtoyr_chg_Cp']] = df_MLS_first[2].str.split(' ', expand=True)

In [69]:
df_MLS_first.head()

,0,1,2,3,4,5,6,7,8,9,10,year,month,benchmark_Cp,yrtoyr_chg_Cp
0,TREB Total,145.6,"$447,400 7.30%",144.7,"$544,800 8.39%",149.2,"$426,900 8.51%",142.0,"$323,500 6.13%",143.7,"$303,300 4.06%",12,2,"$447,400",7.30%
1,City of Toronto,150.5,"$482,600 6.29%",151.6,"$660,000 8.36%",158.5,"$529,300 9.31%",146.5,"$370,200 5.02%",146.7,"$320,100 3.31%",12,2,"$482,600",6.29%
2,Toronto W01,159.2,"$567,800 7.42%",160.1,"$840,100 12.43%",165.2,"$632,500 9.84%",194.8,"$351,500 17.92%",140.6,"$322,800 -3.63%",12,2,"$567,800",7.42%
3,Toronto W02,163.0,"$572,400 13.67%",161.4,"$664,800 11.54%",182.5,"$557,600 17.14%",128.1,"$366,800 4.83%",110.3,"$326,600 -1.52%",12,2,"$572,400",13.67%
4,Toronto W03,146.9,"$375,700 6.06%",147.8,"$403,900 7.26%",150.0,"$386,100 5.71%",-,- -,131.9,"$216,300 -7.83%",12,2,"$375,700",6.06%


In [70]:
df_MLS_first[['benchmark_SFD', 'yrtoyr_chg_SFD']] = df_MLS_first[4].str.split(' ', expand=True)
df_MLS_first[['benchmark_SFA', 'yrtoyr_chg_SFA']] = df_MLS_first[6].str.split(' ', expand=True)
df_MLS_first[['benchmark_Th', 'yrtoyr_chg_Th']] = df_MLS_first[8].str.split(' ', expand=True)
df_MLS_first[['benchmark_Ap', 'yrtoyr_chg_Ap']] = df_MLS_first[10].str.split(' ', expand=True)

In [71]:
df_MLS_first.head()

,0,1,2,3,4,5,6,7,8,9,...,benchmark_Cp,yrtoyr_chg_Cp,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th,benchmark_Ap,yrtoyr_chg_Ap
0,TREB Total,145.6,"$447,400 7.30%",144.7,"$544,800 8.39%",149.2,"$426,900 8.51%",142.0,"$323,500 6.13%",143.7,...,"$447,400",7.30%,"$544,800",8.39%,"$426,900",8.51%,"$323,500",6.13%,"$303,300",4.06%
1,City of Toronto,150.5,"$482,600 6.29%",151.6,"$660,000 8.36%",158.5,"$529,300 9.31%",146.5,"$370,200 5.02%",146.7,...,"$482,600",6.29%,"$660,000",8.36%,"$529,300",9.31%,"$370,200",5.02%,"$320,100",3.31%
2,Toronto W01,159.2,"$567,800 7.42%",160.1,"$840,100 12.43%",165.2,"$632,500 9.84%",194.8,"$351,500 17.92%",140.6,...,"$567,800",7.42%,"$840,100",12.43%,"$632,500",9.84%,"$351,500",17.92%,"$322,800",-3.63%
3,Toronto W02,163.0,"$572,400 13.67%",161.4,"$664,800 11.54%",182.5,"$557,600 17.14%",128.1,"$366,800 4.83%",110.3,...,"$572,400",13.67%,"$664,800",11.54%,"$557,600",17.14%,"$366,800",4.83%,"$326,600",-1.52%
4,Toronto W03,146.9,"$375,700 6.06%",147.8,"$403,900 7.26%",150.0,"$386,100 5.71%",-,- -,131.9,...,"$375,700",6.06%,"$403,900",7.26%,"$386,100",5.71%,-,-,"$216,300",-7.83%


In [72]:
#drop the existing repetitive columns columns 
df_MLS_first.drop(columns=[2, 4, 6, 8, 10], inplace=True)

In [73]:
df_MLS_first.head()

,0,1,3,5,7,9,year,month,benchmark_Cp,yrtoyr_chg_Cp,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th,benchmark_Ap,yrtoyr_chg_Ap
0,TREB Total,145.6,144.7,149.2,142.0,143.7,12,2,"$447,400",7.30%,"$544,800",8.39%,"$426,900",8.51%,"$323,500",6.13%,"$303,300",4.06%
1,City of Toronto,150.5,151.6,158.5,146.5,146.7,12,2,"$482,600",6.29%,"$660,000",8.36%,"$529,300",9.31%,"$370,200",5.02%,"$320,100",3.31%
2,Toronto W01,159.2,160.1,165.2,194.8,140.6,12,2,"$567,800",7.42%,"$840,100",12.43%,"$632,500",9.84%,"$351,500",17.92%,"$322,800",-3.63%
3,Toronto W02,163.0,161.4,182.5,128.1,110.3,12,2,"$572,400",13.67%,"$664,800",11.54%,"$557,600",17.14%,"$366,800",4.83%,"$326,600",-1.52%
4,Toronto W03,146.9,147.8,150.0,-,131.9,12,2,"$375,700",6.06%,"$403,900",7.26%,"$386,100",5.71%,-,-,"$216,300",-7.83%


In [74]:
#change the remaining column names 
col_dict = dict(zip(df_MLS_first.columns[: 6], ['Name', 'index_Cp', 'index_SFD', 'index_SFA',
                                                    'index_Th', 'index_Ap']))
df_MLS_first.rename(columns=col_dict, inplace=True)

In [75]:
df_MLS_first.head()

,Name,index_Cp,index_SFD,index_SFA,index_Th,index_Ap,year,month,benchmark_Cp,yrtoyr_chg_Cp,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th,benchmark_Ap,yrtoyr_chg_Ap
0,TREB Total,145.6,144.7,149.2,142.0,143.7,12,2,"$447,400",7.30%,"$544,800",8.39%,"$426,900",8.51%,"$323,500",6.13%,"$303,300",4.06%
1,City of Toronto,150.5,151.6,158.5,146.5,146.7,12,2,"$482,600",6.29%,"$660,000",8.36%,"$529,300",9.31%,"$370,200",5.02%,"$320,100",3.31%
2,Toronto W01,159.2,160.1,165.2,194.8,140.6,12,2,"$567,800",7.42%,"$840,100",12.43%,"$632,500",9.84%,"$351,500",17.92%,"$322,800",-3.63%
3,Toronto W02,163.0,161.4,182.5,128.1,110.3,12,2,"$572,400",13.67%,"$664,800",11.54%,"$557,600",17.14%,"$366,800",4.83%,"$326,600",-1.52%
4,Toronto W03,146.9,147.8,150.0,-,131.9,12,2,"$375,700",6.06%,"$403,900",7.26%,"$386,100",5.71%,-,-,"$216,300",-7.83%


Let's clean second dataframe 

In [76]:
df_MLS_second.head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month
0,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,"$326,600",3.90%,15,7
1,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,"$341,800",3.83%,15,7
2,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,"$332,100",0.99%,15,7
3,Toronto W02,195,"$700,900",2.86%,"201 $824,200 3.67%",226,"$691,400 3.34%",156,"$431,200 0.71%",141,"$571,500",4.53%,15,7
4,Toronto W03,202,"$523,600",10.75%,"204 $554,800 9.38%",213,"$544,400 13.21%",-,- -,143,"$272,500",6.48%,15,7


In [77]:
columns_MLS = ['Name', 'index_Cp', 'benchmark_Cp', 'yrtoyr_chg_Cp',
              'index_SFD', 'benchmark_SFD', 'yrtoyr_chg_SFD',
              'index_SFA', 'benchmark_SFA', 'yrtoyr_chg_SFA',
              'index_Th', 'benchmark_Th', 'yrtoyr_chg_Th',
              'index_Ap', 'benchmark_Ap', 'yrtoyr_chg_Ap', 'year', 'month']

In [78]:
df_MLS_second[['index_SFD', 'benchmark_SFD', 'yrtoyr_chg_SFD']] = df_MLS_second[4].str.split(' ', expand=True)

In [79]:
df_MLS_second.head()

,0,1,2,3,4,5,6,7,8,9,10,11,year,month,index_SFD,benchmark_SFD,yrtoyr_chg_SFD
0,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,"$326,600",3.90%,15,7,188,"$700,600",10.90%
1,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,"$341,800",3.83%,15,7,195,"$843,300",8.84%
2,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,"$332,100",0.99%,15,7,181,"$932,400",5.79%
3,Toronto W02,195,"$700,900",2.86%,"201 $824,200 3.67%",226,"$691,400 3.34%",156,"$431,200 0.71%",141,"$571,500",4.53%,15,7,201,"$824,200",3.67%
4,Toronto W03,202,"$523,600",10.75%,"204 $554,800 9.38%",213,"$544,400 13.21%",-,- -,143,"$272,500",6.48%,15,7,204,"$554,800",9.38%


In [80]:
df_MLS_second[['benchmark_SFA', 'yrtoyr_chg_SFA']] = df_MLS_second[6].str.split(' ', expand=True)
df_MLS_second[['benchmark_Th', 'yrtoyr_chg_Th']] = df_MLS_second[8].str.split(' ', expand=True)

In [81]:
df_MLS_second.head()

,0,1,2,3,4,5,6,7,8,9,...,11,year,month,index_SFD,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th
0,TREB Total,184,"$561,600",9.39%,"188 $700,600 10.90%",194,"$547,100 10.63%",179,"$392,400 9.46%",163,...,3.90%,15,7,188,"$700,600",10.90%,"$547,100",10.63%,"$392,400",9.46%
1,City of Toronto,184,"$602,000",6.93%,"195 $843,300 8.84%",200,"$663,500 7.89%",183,"$442,800 8.72%",165,...,3.83%,15,7,195,"$843,300",8.84%,"$663,500",7.89%,"$442,800",8.72%
2,Toronto W01,170,"$693,900",2.97%,"181 $932,400 5.79%",197,"$759,000 7.25%",213,"$439,000 0.90%",142,...,0.99%,15,7,181,"$932,400",5.79%,"$759,000",7.25%,"$439,000",0.90%
3,Toronto W02,195,"$700,900",2.86%,"201 $824,200 3.67%",226,"$691,400 3.34%",156,"$431,200 0.71%",141,...,4.53%,15,7,201,"$824,200",3.67%,"$691,400",3.34%,"$431,200",0.71%
4,Toronto W03,202,"$523,600",10.75%,"204 $554,800 9.38%",213,"$544,400 13.21%",-,- -,143,...,6.48%,15,7,204,"$554,800",9.38%,"$544,400",13.21%,-,-


In [82]:
#drop the existing repetitive columns 
df_MLS_second.drop(columns=[4, 6, 8], inplace=True)

In [83]:
df_MLS_second.head()

,0,1,2,3,5,7,9,10,11,year,month,index_SFD,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th
0,TREB Total,184,"$561,600",9.39%,194,179,163,"$326,600",3.90%,15,7,188,"$700,600",10.90%,"$547,100",10.63%,"$392,400",9.46%
1,City of Toronto,184,"$602,000",6.93%,200,183,165,"$341,800",3.83%,15,7,195,"$843,300",8.84%,"$663,500",7.89%,"$442,800",8.72%
2,Toronto W01,170,"$693,900",2.97%,197,213,142,"$332,100",0.99%,15,7,181,"$932,400",5.79%,"$759,000",7.25%,"$439,000",0.90%
3,Toronto W02,195,"$700,900",2.86%,226,156,141,"$571,500",4.53%,15,7,201,"$824,200",3.67%,"$691,400",3.34%,"$431,200",0.71%
4,Toronto W03,202,"$523,600",10.75%,213,-,143,"$272,500",6.48%,15,7,204,"$554,800",9.38%,"$544,400",13.21%,-,-


In [84]:
col_dict_second = dict(zip(df_MLS_second.columns[: 10], ['Name', 'index_Cp', 'benchmark_Cp', 'yrtoyr_chg_Cp',
                                                 'index_SFA', 'index_Th', 'index_Ap', 'benchmark_Ap',
                                                  'yrtoyr_chg_Ap']))
df_MLS_second.rename(columns=col_dict_second, inplace=True)

In [85]:
#change column names for second and third dataframes 
df_MLS_third.columns = columns_MLS

In [86]:
display(df_MLS_second.head())
display(df_MLS_third.head())

,Name,index_Cp,benchmark_Cp,yrtoyr_chg_Cp,index_SFA,index_Th,index_Ap,benchmark_Ap,yrtoyr_chg_Ap,year,month,index_SFD,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th
0,TREB Total,184,"$561,600",9.39%,194,179,163,"$326,600",3.90%,15,7,188,"$700,600",10.90%,"$547,100",10.63%,"$392,400",9.46%
1,City of Toronto,184,"$602,000",6.93%,200,183,165,"$341,800",3.83%,15,7,195,"$843,300",8.84%,"$663,500",7.89%,"$442,800",8.72%
2,Toronto W01,170,"$693,900",2.97%,197,213,142,"$332,100",0.99%,15,7,181,"$932,400",5.79%,"$759,000",7.25%,"$439,000",0.90%
3,Toronto W02,195,"$700,900",2.86%,226,156,141,"$571,500",4.53%,15,7,201,"$824,200",3.67%,"$691,400",3.34%,"$431,200",0.71%
4,Toronto W03,202,"$523,600",10.75%,213,-,143,"$272,500",6.48%,15,7,204,"$554,800",9.38%,"$544,400",13.21%,-,-


,Name,index_Cp,benchmark_Cp,yrtoyr_chg_Cp,index_SFD,benchmark_SFD,yrtoyr_chg_SFD,index_SFA,benchmark_SFA,yrtoyr_chg_SFA,index_Th,benchmark_Th,yrtoyr_chg_Th,index_Ap,benchmark_Ap,yrtoyr_chg_Ap,year,month
0,TREB Total,273.0,"$828,200",8.72%,262.9,"$970,700",7.39%,273.1,"$767,800",7.73%,280.4,"$619,000",8.81%,282.6,"$564,000",11.26%,20,1
1,City of Toronto,281.8,"$913,700",8.76%,265.1,"$1,163,300",6.81%,281.6,"$938,700",6.99%,278.0,"$661,800",4.71%,289.7,"$595,000",10.61%,20,1
2,Toronto W01,264.1,"$1,064,400",6.53%,249.6,"$1,302,400",5.09%,267.8,"$1,032,600",5.60%,271.9,"$575,500",7.64%,268.0,"$605,900",7.37%,20,1
3,Toronto W02,296.0,"$1,053,400",5.60%,283.4,"$1,198,800",5.20%,314.1,"$962,700",1.19%,276.7,"$652,000",2.67%,306.5,"$657,900",12.81%,20,1
4,Toronto W03,302.4,"$778,300",6.89%,304.5,"$827,800",6.92%,297.2,"$761,200",6.64%,267.5,"$545,700",1.10%,303.3,"$518,200",10.94%,20,1


In [87]:
df_MLS_fourth.columns = ['Name', 'index_Cp', 'benchmark_Cp', 'yrtoyr_chg_Cp',
              'index_SFD', 'benchmark_SFD', 'yrtoyr_chg_SFD',
              'index_SFA', 'benchmark_SFA', 'yrtoyr_chg_SFA',
              'index_Th', 'benchmark_Th', 'yrtoyr_chg_Th',
              'index_Ap', 'benchmark_Ap', 'year', 'month']

In [88]:
df_MLS_fourth.head()

,Name,index_Cp,benchmark_Cp,yrtoyr_chg_Cp,index_SFD,benchmark_SFD,yrtoyr_chg_SFD,index_SFA,benchmark_SFA,yrtoyr_chg_SFA,index_Th,benchmark_Th,yrtoyr_chg_Th,index_Ap,benchmark_Ap,year,month
0,TRREB Total,331.9,"$1,007,600",16.54%,337.5,"$1,246,900",23.58%,348.3,"$978,500",21.91%,338.1,"$742,800",16.51%,303.1,"$607,500",21,3
1,City of Toronto,317.4,"$1,029,200",7.45%,325.0,"$1,426,100",17.84%,339.8,"$1,132,600",15.74%,317.4,"$756,300",9.71%,306.0,"$630,300",21,3
2,Toronto W01,299.6,"$1,207,400",10.55%,307.2,"$1,603,000",18.75%,326.0,"$1,257,100",18.33%,290.2,"$614,200",5.57%,291.7,"$659,500",21,3
3,Toronto W02,357.2,"$1,269,700",13.50%,347.4,"$1,469,500",18.73%,387.3,"$1,187,200",18.77%,374.9,"$723,200",4.55%,334.4,"$717,800",21,3
4,Toronto W03,370.9,"$956,000",18.80%,382.4,"$1,039,600",20.82%,373.0,"$955,400",21.34%,266.0,"$644,800",5.06%,328.9,"$562,000",21,3


In [89]:
#combine four dataframes to get MLS data 
df_MLS = pd.concat([df_MLS_first, df_MLS_second, df_MLS_third, df_MLS_fourth], ignore_index=True)

In [90]:
df_MLS.tail()

,Name,index_Cp,index_SFD,index_SFA,index_Th,index_Ap,year,month,benchmark_Cp,yrtoyr_chg_Cp,benchmark_SFD,yrtoyr_chg_SFD,benchmark_SFA,yrtoyr_chg_SFA,benchmark_Th,yrtoyr_chg_Th,benchmark_Ap,yrtoyr_chg_Ap
4482,Toronto E06,448.3,456.9,452.7,431.2,405.4,22,3,"$1,246,700",28.90%,"$1,305,100",26.99%,"$1,076,700",25.89%,"$953,000",34.00%,"$834,600",NaN
4483,Toronto E07,443.3,449.4,438.6,381.8,447.3,22,3,"$1,086,600",36.65%,"$1,473,000",36.80%,"$1,146,800",37.02%,"$827,100",18.72%,"$766,000",NaN
4484,Toronto E08,447.0,438.7,393.7,427.4,466.1,22,3,"$1,030,500",29.42%,"$1,367,400",29.87%,"$1,020,700",32.20%,"$783,600",39.22%,"$636,000",NaN
4485,Toronto E09,455.7,458.9,433.4,492.5,448.8,22,3,"$1,088,100",38.85%,"$1,303,000",36.90%,"$1,060,000",31.85%,"$886,600",41.77%,"$841,800",NaN
4486,Toronto E10,471.1,454.4,459.7,548.1,418.8,22,3,"$1,321,200",37.95%,"$1,455,900",37.12%,"$1,192,000",36.57%,"$894,100",44.31%,"$675,700",NaN


In [91]:
#save data to a csv file 
df_MLS.to_csv('../data/processed/housing_MLS.csv')

Now that we have monthly sales, housing prices, and MLS index data for each neighborhood and month over last 10 years. In next notebook we will clean this data and do some exploratory data analysis.   